# data analysis

(ignore it for now)

In [ ]:
# numpy and pandas

import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel('/content/data.xlsx')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: '/content/data.xlsx'

In [ ]:
print(type(data))
print(type([12,34]))

In [ ]:
missing_data=data.isnull().sum()
missing_data[missing_data > 0]

In [ ]:
summary=data.describe()
summary

In [ ]:
# matplotlib, seaborn

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# histogram

data.hist(bins=20, figsize=(10,10))
plt.suptitle('Histogram of Numeric Columns')
plt.show()

In [ ]:
# boxplot

plt.figure(figsize=(10,10))
sns.boxplot(data=data)
plt.suptitle('Boxplot of Numeric Columns')
plt.yscale('log')
plt.xticks(rotation=90)
plt.show()

In [ ]:
# scatter plot

plt.figure(figsize=(10,10))
sns.scatterplot(x=data["MIN"],y=data["FG%"])
plt.suptitle('MIN vs FG%')
plt.xlabel('MIN')
plt.ylabel('FG%')
plt.show()

In [ ]:
# correlation heatmap

plt.figure(figsize=(30,30))
filtered_data=data.select_dtypes(include=[np.number])
sns.heatmap(filtered_data.corr(), annot=True, cmap="coolwarm",fmt=".2f")
plt.suptitle('Correlation Heatmap')
plt.show()

# pearson correlation coefficient

In [ ]:
# PCA (principal component analysis)

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


filtered_data=data.select_dtypes(include=[np.number])

filtered_data=filtered_data.dropna() # drop the missing value

target_column='Salary'
target=filtered_data[target_column]

scaler=StandardScaler() # create an instance from a class
scaled_data=scaler.fit_transform(filtered_data) # call a method of the instance

pca=PCA(n_components=None) # the number of new features = the number of original features
pca.fit(scaled_data)

features_pca= pca.transform(scaled_data)

pca_df = pd.DataFrame(features_pca,columns=[f'PC{i+1}' for i in range(features_pca.shape[1])])

pca_df[target_column]=target.values

correlation_with_target = pca_df.corr()[target_column].drop(target_column)

print(correlation_with_target)

# print(pca_df)

correlation_with_original_features=pd.DataFrame(np.corrcoef(scaled_data.T, features_pca.T)[:scaled_data.shape[1],scaled_data.shape[1]:], index=filtered_data.columns, columns=[f'PC{i+1}' for i in range(features_pca.shape[1])])

print(correlation_with_original_features)

plt.figure(figsize=(10,10))
plt.plot(range(1,len(pca.explained_variance_ratio_)+1),np.cumsum(pca.explained_variance_ratio_))
plt.suptitle('Cumulative Explained Variance')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.show()

# installations

(restart the runtime after running it)



In [ ]:
!pip install catboost ngboost
!pip install lightgbm --install-option=--gpu
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.8.* dask-cudf-cu12==24.8.* cuml-cu12==24.8.* \
    cugraph-cu12==24.8.* cuspatial-cu12==24.8.* cuproj-cu12==24.8.* \
    cuxfilter-cu12==24.8.* cucim-cu12==24.8.* pylibraft-cu12==24.8.* \
    raft-dask-cu12==24.8.* cuvs-cu12==24.8.*

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 4.8 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=d0bfc992780b1aa3d5c9e7e8e25f53f02849e404f8fa841052d9bd1cb8ef1e77
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --install-option
Looking in indexes: https://pypi.org/simple, https://pypi.nvidi

# data processing

(running it once each time before running modeling)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_excel('/content/data_raw.xlsx')
data.head()

,PLAYER,Draft Year,S1_TEAM,S1_AGE,S1_GP,S1_W,S1_,S1_MIN,S1_PTS,S1_GM,...,S4_TeamL,S4_PlayOff,S4_Ranking,S4_Ranking Difference,S4_Salary,S4_Adjusted Salary,S4_Salary Cap Proportion,C2_Salary,C2_Adjusted Salary,C2_Salary Cap Proportion
0,Karl-Anthony Towns,2015,MIN,20,82,29,53,32.0,18.3,7.6,...,46.0,0.0,11.0,-3.0,7839435.0,9492010.0,0.076956,27285000.0,32500982.0,0.250000
1,Emmanuel Mudiay,2015,DEN,20,68,29,39,30.4,12.8,4.8,...,65.0,0.0,15.0,-4.0,4294479.0,5199767.0,0.042157,1737145.0,2069229.0,0.015917
2,Jahlil Okafor,2015,PHI,20,53,6,47,30.0,17.5,7.5,...,49.0,0.0,13.0,-7.0,1567007.0,1897336.0,0.015383,1702486.0,2027944.0,0.015599
3,Justise Winslow,2015,MIA,20,78,46,32,28.6,6.4,2.5,...,43.0,0.0,10.0,-4.0,3448926.0,4175969.0,0.033856,13000000.0,15485166.0,0.119113
4,Kristaps Porzingis,2015,NYK,20,72,28,44,28.4,14.3,5.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
summary=data.describe()
summary

,Draft Year,S1_AGE,S1_GP,S1_W,S1_,S1_MIN,S1_PTS,S1_GM,S1_FGA,S1_FG%,...,S4_TeamL,S4_PlayOff,S4_Ranking,S4_Ranking Difference,S4_Salary,S4_Adjusted Salary,S4_Salary Cap Proportion,C2_Salary,C2_Adjusted Salary,C2_Salary Cap Proportion
count,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,169.000000,...,152.000000,152.000000,152.000000,152.000000,1.500000e+02,1.500000e+02,150.000000,1.320000e+02,1.320000e+02,132.000000
mean,2017.520710,20.816568,52.538462,23.183432,29.355030,18.760947,7.608284,2.869231,6.565680,43.231361,...,41.861842,0.453947,8.914474,-0.559211,5.018457e+06,5.533272e+06,0.042930,1.419972e+07,1.520633e+07,0.115243
std,1.722017,1.251936,21.932255,11.585426,15.462069,8.238976,4.688073,1.765071,3.849058,8.554783,...,12.419302,0.499521,4.571643,4.305754,2.774879e+06,2.952932e+06,0.022841,1.109363e+07,1.170726e+07,0.088515
min,2015.000000,19.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,...,18.000000,0.000000,1.000000,-14.000000,7.725000e+04,9.353400e+04,0.000758,1.689830e+05,1.999950e+05,0.001548
25%,2016.000000,20.000000,38.000000,15.000000,17.000000,12.100000,3.800000,1.500000,3.600000,39.900000,...,33.750000,0.000000,5.000000,-3.000000,3.457191e+06,4.014091e+06,0.031365,4.253964e+06,4.554108e+06,0.034145
50%,2018.000000,20.000000,57.000000,22.000000,30.000000,19.200000,6.400000,2.500000,5.700000,42.700000,...,40.000000,0.000000,9.000000,0.000000,4.379526e+06,4.773740e+06,0.037707,1.281364e+07,1.373788e+07,0.103891
75%,2019.000000,22.000000,69.000000,30.000000,42.000000,25.400000,10.300000,3.700000,8.800000,46.900000,...,50.000000,1.000000,13.000000,1.000000,5.887899e+06,6.359463e+06,0.049330,2.249107e+07,2.349982e+07,0.180740
max,2020.000000,24.000000,82.000000,53.000000,63.000000,34.200000,22.500000,8.800000,17.200000,70.600000,...,68.000000,1.000000,15.000000,9.000000,1.353482e+07,1.454397e+07,0.112596,4.217640e+07,4.217640e+07,0.300000


In [ ]:
missing_data=data.isnull().sum()
missing_data[missing_data > 0]

,0
S2_TEAM,1
S2_AGE,1
S2_GP,1
S2_W,1
S2_L,1
...,...
S4_Adjusted Salary,19
S4_Salary Cap Proportion,19
C2_Salary,37
C2_Adjusted Salary,37


In [ ]:
# encoding

from sklearn.preprocessing import LabelEncoder

one_hot_encoder_instance = LabelEncoder()

for column in data.select_dtypes(include=['object']).columns:
  data[column]=one_hot_encoder_instance.fit_transform(data[column])

print(data)

     PLAYER  Draft Year  S1_TEAM  S1_AGE  S1_GP  S1_W  S1_  S1_MIN  S1_PTS  \
0        84        2015       17      20     82    29   53    32.0    18.3   
1        44        2015        7      20     68    29   39    30.4    12.8   
2        58        2015       22      20     53     6   47    30.0    17.5   
3        83        2015       15      20     78    46   32    28.6     6.4   
4        94        2015       19      20     72    28   44    28.4    14.3   
..      ...         ...      ...     ...    ...   ...  ...     ...     ...   
164     123        2020       19      23     62    36   26    11.0     4.1   
165     166        2020        7      20     42    30   12     9.5     3.2   
166      75        2020        6      20     39    20   19    11.4     2.6   
167      60        2020       23      21     27    19    8     5.8     2.0   
168     161        2020       28      21     15    13    2     3.8     1.1   

     S1_GM  ...  S4_TeamL  S4_PlayOff  S4_Ranking  S4_Ranking D

In [ ]:
filtered_data=data.select_dtypes(include=[np.number])
filtered_data=filtered_data.dropna() # drop the missing value

missing_data_new=filtered_data.isnull().sum()
missing_data_new[missing_data > 0]

,0
S2_TEAM,0
S2_AGE,0
S2_GP,0
S2_W,0
S2_L,0
...,...
S4_Adjusted Salary,0
S4_Salary Cap Proportion,0
C2_Salary,0
C2_Adjusted Salary,0


In [ ]:
print("before normalization")
filtered_data.head()

before normalization


,PLAYER,Draft Year,S1_TEAM,S1_AGE,S1_GP,S1_W,S1_,S1_MIN,S1_PTS,S1_GM,...,S4_TeamL,S4_PlayOff,S4_Ranking,S4_Ranking Difference,S4_Salary,S4_Adjusted Salary,S4_Salary Cap Proportion,C2_Salary,C2_Adjusted Salary,C2_Salary Cap Proportion
0,84,2015,17,20,82,29,53,32.0,18.3,7.6,...,46.0,0.0,11.0,-3.0,7839435.0,9492010.0,0.076956,27285000.0,32500982.0,0.250000
1,44,2015,7,20,68,29,39,30.4,12.8,4.8,...,65.0,0.0,15.0,-4.0,4294479.0,5199767.0,0.042157,1737145.0,2069229.0,0.015917
2,58,2015,22,20,53,6,47,30.0,17.5,7.5,...,49.0,0.0,13.0,-7.0,1567007.0,1897336.0,0.015383,1702486.0,2027944.0,0.015599
3,83,2015,15,20,78,46,32,28.6,6.4,2.5,...,43.0,0.0,10.0,-4.0,3448926.0,4175969.0,0.033856,13000000.0,15485166.0,0.119113
5,21,2015,13,20,80,17,63,28.2,13.2,4.9,...,40.0,1.0,7.0,5.0,7019698.0,8499470.0,0.068909,27285000.0,32500982.0,0.250000


In [ ]:
from sklearn.preprocessing import StandardScaler

# from the 2nd to the 4th to the last
input_columns=filtered_data.columns[1:-3]

scaler=StandardScaler() # create an instance from a class
scaled_data=scaler.fit_transform(filtered_data[input_columns]) # call a method of the instance
filtered_data[input_columns]=scaled_data
scaled_data=filtered_data
scaled_data.head()

,PLAYER,Draft Year,S1_TEAM,S1_AGE,S1_GP,S1_W,S1_,S1_MIN,S1_PTS,S1_GM,...,S4_TeamL,S4_PlayOff,S4_Ranking,S4_Ranking Difference,S4_Salary,S4_Adjusted Salary,S4_Salary Cap Proportion,C2_Salary,C2_Adjusted Salary,C2_Salary Cap Proportion
0,84,-1.496202,0.304157,-0.670215,1.266838,0.310916,1.406611,1.474513,2.133844,2.556195,...,0.361266,-0.930415,0.488233,-0.648274,1.017513,1.349627,1.504596,27285000.0,32500982.0,0.250000
1,44,-1.496202,-0.797860,-0.670215,0.523887,0.310916,0.445436,1.272373,0.946552,0.942727,...,1.904446,-0.930415,1.366351,-0.892353,-0.394250,-0.240738,-0.157580,1737145.0,2069229.0,0.015917
2,58,-1.496202,0.855165,-0.670215,-0.272132,-1.802286,0.994679,1.221838,1.961147,2.498572,...,0.604926,-0.930415,0.927292,-1.624591,-1.480453,-1.464356,-1.436450,1702486.0,2027944.0,0.015599
3,83,-1.496202,0.083753,-0.670215,1.054566,1.872848,-0.035152,1.044965,-0.435024,-0.382623,...,0.117607,-0.930415,0.268704,-0.892353,-0.730987,-0.620076,-0.554047,13000000.0,15485166.0,0.119113
5,21,-1.496202,-0.136650,-0.670215,1.160702,-0.791624,2.093165,0.994430,1.032901,1.000351,...,-0.126053,1.074789,-0.389884,1.304359,0.691056,0.981870,1.120234,27285000.0,32500982.0,0.250000


In [ ]:
# for testing the index of the feature
print(filtered_data.columns[148])
print(filtered_data.columns[-3])
print(filtered_data.columns[148:-3])

S4_TEAM
C2_Salary
Index(['S4_TEAM', 'S4_AGE', 'S4_GP', 'S4_W', 'S4_L', 'S4_MIN', 'S4_PTS',
       'S4_FGM', 'S4_FGA', 'S4_FG%', 'S4_3PM', 'S4_3PA', 'S4_3P%', 'S4_FTM',
       'S4_FTA', 'S4_FT%', 'S4_OREB', 'S4_DREB', 'S4_REB', 'S4_AST', 'S4_TOV',
       'S4_STL', 'S4_BLK', 'S4_PF', 'S4_FP', 'S4_DD2', 'S4_TD3', 'S4_+/-',
       'S4_OFFRTG', 'S4_DEFRTG', 'S4_NETRTG', 'S4_AST%', 'S4_AST/TO',
       'S4_AST RATIO', 'S4_OREB%', 'S4_DREB%', 'S4_REB%', 'S4_TO RATIO',
       'S4_EFG%', 'S4_TS%', 'S4_USG%', 'S4_TeamW', 'S4_TeamL', 'S4_PlayOff',
       'S4_Ranking', 'S4_Ranking Difference', 'S4_Salary',
       'S4_Adjusted Salary', 'S4_Salary Cap Proportion'],
      dtype='object')


In [ ]:
print(filtered_data.columns[1:-3:50]) # for a single feature

Index(['Draft Year', 'S2_AGE', 'S3_GP', 'S4_W'], dtype='object')


In [ ]:
filtered_data.iloc[:,[100,120,148]] # select multiple features

,S3_AGE,S3_STL,S4_TEAM
0,-0.670215,0.004625,0.425369
1,-0.670215,-0.573503,0.652112
2,-0.670215,-2.018822,0.538741
3,-0.670215,0.004625,0.198627
5,-0.670215,0.004625,-1.388572
...,...,...,...
163,-0.670215,-0.284439,-1.501943
164,1.700829,-1.440694,-0.254859
165,-0.670215,-1.440694,-0.708344
166,-0.670215,-0.284439,-0.821715


In [ ]:
# select the input data (from the 2nd to the 4th to last columns)
scaled_data=np.asarray(scaled_data)
# input_data= scaled_data[:, 1:-3] # select all features
input_data= scaled_data[:, 148:-3] # select the features for S4
# input_data= scaled_data[:, 1:-3:50] # select features with equal spacing (useful for a single feature across seasons)
# input_data= scaled_data[:,[100,120,148]] # select multiple features
print(input_data)

# select the output data (the 3rd to last column)

# output_data = scaled_data [:,-3] # select data for output data "-3"
output_data = scaled_data [:,-2] # for adjusted salaries
print(output_data)

[[ 0.42536922 -0.67021523  1.2200876  ...  1.01751297  1.34962667
   1.50459602]
 [ 0.65211188 -0.67021523 -0.00217291 ... -0.39424977 -0.24073763
  -0.1575798 ]
 [ 0.53874055 -0.67021523 -0.00217291 ... -1.48045319 -1.46435623
  -1.43644983]
 ...
 [-0.70834406 -0.67021523 -0.07007627 ... -0.38954967 -0.57179257
  -0.65901181]
 [-0.82171539 -0.67021523 -0.13797963 ... -0.20673189 -0.40170214
  -0.49781006]
 [-0.25485875  0.12013292  0.13363382 ... -0.09584681 -0.29853662
  -0.40003582]]
[32500982.  2069229.  2027944. 15485166. 32500982. 32500982.  4315596.
 21440999.  2593745.  2977916.  5678291. 15160303.  2069229. 17867499.
  6551416.   233811. 12049841.   233811. 18611978. 11284736. 23697947.
  5317708. 29507409. 32292459. 32292459. 19177476. 29629669.  2158751.
 23433780. 14498172.  7684622. 36167554.  2055953. 15889872.  5641860.
  9588738.  2055953.  1760235. 20892639. 31559650. 14598731. 31559650.
 17620566.  2009297. 31559650.  5620904. 25828525. 13475752.   346431.
  2009297. 

In [ ]:
# split into training and testing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(input_data, output_data, test_size=0.33, random_state=0)


In [ ]:
# import xgboost as xgb
# from sklearn.model_selection import KFold
# from sklearn.metrics import mean_squared_error
# import numpy as np

# # Best parameters from previous tuning
# best_params = {
#     'learning_rate': 0.1,
#     'n_estimators': 1000,
#     'max_depth': 7,
#     'min_child_weight': 1,
#     'gamma': 0.3,
#     'subsample': 0.7,
#     'colsample_bytree': 0.8,
#     'reg_alpha': 10,
#     'reg_lambda': 10,
#     'scale_pos_weight': 5,
#     'device': 'cuda'  # Using GPU
# }

# # Initialize the model with the best parameters
# model = xgb.XGBRegressor(**best_params)

# # Define KFold cross-validation
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Lists to store results
# rmse_list = []

# # Cross-validation loop
# for train_index, test_index in kf.split(input_data):
#     # Split the data into train and test sets
#     X_train, X_test = input_data[train_index], input_data[test_index]
#     y_train, y_test = output_data[train_index], output_data[test_index]

#     # Fit the model on the training data
#     model.fit(X_train, y_train)

#     # Predict on the test data
#     predicted_y_test = model.predict(X_test)

#     # Calculate RMSE for the current fold
#     rmse = np.sqrt(mean_squared_error(y_test, predicted_y_test))
#     rmse_list.append(rmse)

#     print(f"Fold RMSE: {rmse}")

# # Calculate average RMSE across all folds
# average_rmse = np.mean(rmse_list)

# # Print the cross-validation result
# print(f"Average RMSE across all folds: {average_rmse}")

In [ ]:
print(y_test)

[ 2593745. 14989221.  2009297. 32292459. 12871147. 10500000. 18529159.
  2165000. 21440999. 35147000.  5678291.  2026613.   624425.   346431.
 20089286.  2027944.  6250000. 27000000. 29507409.  5620904. 17606711.
 15860000. 25679348. 17195852. 12049841.  7684622.  2237691. 17867499.
 15889872.  4633612. 29629669. 11000000. 31831634. 38197960.  8571429.
 22984191. 15485166. 31831634.  4315596. 38197960.  6000000. 27955357.]


In [ ]:
import random
import cuml
from cuml.ensemble import RandomForestRegressor as cuRF
from cuml.linear_model import Ridge as cuRidge
from cuml.svm import SVR as cuSVR
from cuml.metrics import mean_squared_error
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import cudf
import cupy as cp  # Import CuPy to handle GPU-based arrays
import xgboost as xgb  # Import XGBoost for GPU-based gradient boosting

from cuml.linear_model import ElasticNet as cuElasticNet
from cuml.linear_model import Lasso as cuLasso
from cuml.neighbors import KNeighborsRegressor as cuKNN
from sklearn.ensemble import GradientBoostingRegressor


# modeling


1.   (First, debug this section of code)
2.   (Then, adjust the n_iter in the function trial to search exhaustively without consuming too much time)
3.   (Find the best configuraiton using the code, run the code with as much time as possible)



In [ ]:
from tqdm import tqdm

models_and_params = {
    "XGBoostRegressor": {
        "model": xgb.XGBRegressor(device='cuda'),  # XGBoost with GPU support
        "params": {
            "learning_rate": [0.01, 0.05, 0.1, 0.3],
            "n_estimators": [100, 200, 500, 1000],
            "max_depth": [3, 5, 7, 9],
            "min_child_weight": [1, 3, 5, 7],
            "gamma": [0, 0.1, 0.2, 0.3],
            "subsample": [0.6, 0.7, 0.8, 0.9],
            "colsample_bytree": [0.6, 0.7, 0.8, 0.9],
            "reg_alpha": [0, 0.1, 1, 10],
            "reg_lambda": [0.1, 1, 5, 10],
            "scale_pos_weight": [1, 3, 5]
        }
    },
    "GradientBoostingRegressor": {
        "model": GradientBoostingRegressor(),  # Sklearn's Gradient Boosting without GPU
        "params": {
            "learning_rate": [0.01, 0.05, 0.1, 0.2],
            "n_estimators": [100, 200, 500],
            "max_depth": [3, 5, 7],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
            "subsample": [0.6, 0.8, 1.0],
            "max_features": ["sqrt", "log2", None],
        }
    },
    "cuRandomForestRegressor": {
        "model": cuRF(),
        "params": {
            "n_estimators": [100, 200, 500, 1000],
            "max_depth": [None, 10, 20, 30],
            "min_samples_split": [2, 5, 10],
            "min_samples_leaf": [1, 2, 4],
            "max_features": ["sqrt", "log2", 0.3, 0.5, 0.7, 1.0],
            "bootstrap": [True, False],
            "max_samples": [None, 0.5, 0.7, 0.9],
            "min_impurity_decrease": [0.0, 0.1, 0.2],
            "max_leaf_nodes": [None, 50, 100, 200]
        }
    },
    "cuRidge": {
        "model": cuRidge(),
        "params": {
            "alpha": [0.001, 0.01, 0.1, 1.0, 10.0, 100.0],
            "fit_intercept": [True, False],
            "solver": ['eig', 'svd'],
        }
    },
    "cuSVR": {
        "model": cuSVR(),
        "params": {
            "C": [0.1, 1, 10, 100],
            "epsilon": [0.01, 0.1, 0.2, 0.5],
            "kernel": ['rbf', 'linear', 'poly'],
            "gamma": ['scale', 'auto', 0.1, 1],
            "tol": [1e-4, 1e-3, 1e-2],
            "max_iter": [-1, 1000, 5000, 10000]
        }
    },
    "cuElasticNet": {
        "model": cuElasticNet(),
        "params": {
            "alpha": [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
            "l1_ratio": [0.1, 0.3, 0.5, 0.7, 0.9],
            "fit_intercept": [True, False],
            "tol": [1e-5, 1e-4, 1e-3],
            "max_iter": [100, 1000, 5000, 10000],
            "selection": ['cyclic', 'random']
        }
    },
    "cuLasso": {
        "model": cuLasso(),
        "params": {
            "alpha": [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
            "fit_intercept": [True, False],
            "max_iter": [100, 1000, 5000, 10000],
            "tol": [1e-5, 1e-4, 1e-3],
            "selection": ['cyclic', 'random']
        }
    },
    "cuKNeighborsRegressor": {
        "model": cuKNN(),
        "params": {
            "n_neighbors": [3, 5, 7, 9, 11, 13, 15],
            "weights": ["uniform", "distance"],
            # "algorithm": ["ball_tree", "kd_tree", "brute"],
            "p": [1, 2, 3],
            "metric": ["minkowski", "euclidean", "manhattan"]
        }
    }
}

def trial(X_train, X_test, y_train, y_test, models_and_params, n_iter=1000):
    best_score = float('inf')
    best_model = None
    best_params = None
    total_search_count = 0  # Counter for the number of hyperparameter searches

    # Convert numpy data to CuPy arrays
    X_train = cp.array(X_train).astype(cp.float32)
    X_test = cp.array(X_test).astype(cp.float32)
    y_train = cp.array(y_train).astype(cp.float32)
    y_test = cp.array(y_test).astype(cp.float32)

    # Loop through each model and its parameters
    for model_name, model_info in models_and_params.items():
        param_grid = model_info["params"]

        print(f"Searching the model: {model_name}")

        # Randomly sample hyperparameters for the given model
        for params in tqdm(param_random_search(param_grid, n_iter)):
            total_search_count += 1  # Increment the counter for each search attempt

            # Create a new instance of the model for each set of hyperparameters
            model = model_info["model"]
            model.set_params(**params)

            try:
                # Handle specific cases, especially for cuRandomForestRegressor
                if model_name == "cuRandomForestRegressor":
                    # Ensure that max_samples is in the valid range (0, 1]
                    if 'max_samples' in params and (params['max_samples'] is None or not (0 < params['max_samples'] <= 1)):
                        print(f"Skipping invalid 'max_samples' value: {params['max_samples']}")
                        continue

                    # Ensure that params are valid (especially for None values)
                    params = {k: v if v is not None else 0 for k, v in params.items()}
                    model.set_params(**params)

                # For XGBoost
                if model_name == "XGBoostRegressor":
                    model.fit(X_train, y_train)

                # For other models
                else:
                    model.fit(X_train, y_train)

                # Predictions
                predicted_y_test = model.predict(X_test)

                # Compute RMSE
                score = cp.sqrt(mean_squared_error(y_test, predicted_y_test))

                # Save the model if it has the best score
                if score < best_score:
                    best_score = score
                    best_model = model_name
                    best_params = params
            except Exception as e:
                print(f"Error with {model_name}: {str(e)}")
                continue

    print(f"Total Search Attempts: {total_search_count}")
    print(f"Best Model: {best_model}")
    print(f"Best Parameters: {best_params}")
    print(f"Best RMSE Score: {best_score}")

    return best_model, best_params, best_score

# Helper function to create a random sample of hyperparameters
def param_random_search(param_grid, n_iter):
    """
    Generates a random sample of n_iter combinations of hyperparameters
    from the param_grid.
    """
    from itertools import product
    import random
    all_combinations = list(product(*param_grid.values()))  # All possible combinations
    random_combinations = random.sample(all_combinations, min(n_iter, len(all_combinations)))  # Randomly select n_iter samples

    # Create the dictionaries for each random combination
    for combination in random_combinations:
        yield dict(zip(param_grid.keys(), combination))

# Running the trial function with random search
best_model, best_params, best_score = trial(X_train, X_test, y_train, y_test, models_and_params, n_iter=10)


Searching the model: XGBoostRegressor


10it [00:08,  1.24it/s]


Searching the model: GradientBoostingRegressor


10it [00:00, 839.55it/s]


Error with GradientBoostingRegressor: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.
Error with GradientBoostingRegressor: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.
Error with GradientBoostingRegressor: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.
Error with GradientBoostingRegressor: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.
Error with GradientBoostingRegressor: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.
Error with GradientBoostingRegressor: Implicit conversion to a NumPy array is not allowed. Please use `.get()` to construct a NumPy array explicitly.
Error with GradientBoostingRegressor: Implicit conversion to a NumPy array is not allowed. Please us

...

Searching the model: cuLasso


10it [00:19,  1.91s/it]


Searching the model: cuKNeighborsRegressor


10it [00:00, 226.72it/s]

Total Search Attempts: 80
Best Model: cuKNeighborsRegressor
Best Parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 2, 'metric': 'euclidean'}
Best RMSE Score: 4286652.0


In [ ]:
ｍodel=xgb.XGBRegressor(device='cuda')
model.fit(X_train, y_train)
predicted_y_test = model.predict(X_test)
score = cp.sqrt(mean_squared_error(y_test, predicted_y_test))
score

array(6057466.72826145)

# testing
(ignore it for now)

In [ ]:
y_test_reshaped=y_test.reshape(-1,1)
predicted_y_test_y_test_reshaped=predicted_y_test.reshape(-1,1)

stacked_results=np.hstack((y_test_reshaped,predicted_y_test_y_test_reshaped))

stacked_results_df=pd.DataFrame(stacked_results, columns=["actual","predicted"])
stacked_results_df=stacked_results_df.applymap(lambda x: f"{x:.2e}")
stacked_results_df